In [1]:
# ВСЕ ЗНАЧИМЫЕ МОДЕЛИ В САМОМ КОНЦЕ!!!!
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt


from datetime import datetime 
from pandas import Series, DataFrame 
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, mean_squared_error
 
import lightgbm as lgb 
import xgboost as xgb 

%matplotlib inline

RANDOM_STATE = 42

# ВСЯКИЕ ФУНКЦИИ

In [2]:
def date_features(df):
    df["quarter"] = df.date.apply(lambda s: s.quarter)
    df["month"] = df.date.apply(lambda s: s.month)
    df["week"] = df.date.apply(lambda s: s.week)
    df["day"] = df.date.apply(lambda s: s.day)
    df["dayofweek"] = df.date.apply(lambda s: s.dayofweek)
    df["year"] = df.date.apply(lambda s: s.year)
    
    return df

def show_data(df):
    for col in df.columns:
        if (col == "id") | (col == "price") | (col == "street_id"):
            pass
        else:
            print("{0}: {1}".format(col, df[col].unique()))
    print(df.columns)
    
def delete_nan(df):
    df.build_tech[df.build_tech.isnull()] = df["build_tech"].median()
    df.metro_dist[df.metro_dist.isnull()] = df["metro_dist"].max()
    if ("g_lift" in df.columns):
        df.g_lift[df.g_lift.isnull()] = df["g_lift"].median()
    return df

def make_submission(ids, preds, output_path='submission.csv'):
    subm = pd.DataFrame()
    subm['id'] = ids
    subm['price'] = preds
    subm.to_csv(output_path, index=False) 

def corr(df):
    s1 = pd.Series(df["price"])
    for col in df.columns:
        if (col != "date"):
            s2 = pd.Series(df[col])
            c = s1.corr(s2 )
            print("corr {0} with price: {1}".format(col, c))

# ПОДГОТОВКА ОБЩИХ ДАННЫХ. АНАЛИЗ

In [3]:
train = pd.read_csv("Train.csv")
train.head()

,id,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,...,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,price
0,0,2011-01-01,511,1.0,5,67,2,1,NaN,1.0,...,0,0,0,0,0,0,0,0,0,18473000
1,1,2011-01-01,299,0.0,3,45,2,0,20.0,0.0,...,0,0,0,0,0,0,0,0,0,4157000
2,2,2011-01-01,217,NaN,1,61,3,1,10.0,1.0,...,0,0,0,0,0,0,0,0,0,4530000
3,3,2011-01-01,212,NaN,2,51,2,1,15.0,0.0,...,0,0,0,0,0,0,0,0,0,7981000
4,4,2011-01-01,160,1.0,3,60,3,1,25.0,0.0,...,0,0,0,0,0,0,0,0,0,11888000


In [4]:
test = pd.read_csv("Test.csv")

In [5]:
train.date = pd.to_datetime(train.date)
test.date = pd.to_datetime(test.date)

In [6]:
train = date_features(train)
test = date_features(test)

train.head()

,id,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,...,kw11,kw12,kw13,price,quarter,month,week,day,dayofweek,year
0,0,2011-01-01,511,1.0,5,67,2,1,NaN,1.0,...,0,0,0,18473000,1,1,52,1,5,2011
1,1,2011-01-01,299,0.0,3,45,2,0,20.0,0.0,...,0,0,0,4157000,1,1,52,1,5,2011
2,2,2011-01-01,217,NaN,1,61,3,1,10.0,1.0,...,0,0,0,4530000,1,1,52,1,5,2011
3,3,2011-01-01,212,NaN,2,51,2,1,15.0,0.0,...,0,0,0,7981000,1,1,52,1,5,2011
4,4,2011-01-01,160,1.0,3,60,3,1,25.0,0.0,...,0,0,0,11888000,1,1,52,1,5,2011


In [7]:
train.corr()

,id,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,n_photos,...,kw11,kw12,kw13,price,quarter,month,week,day,dayofweek,year
id,1.000000,-0.004561,-0.002651,0.000376,0.001707,0.002140,-0.001096,-0.000467,0.000224,0.003981,...,0.004462,0.002672,-0.000806,0.034704,0.137598,0.169413,-0.019723,NaN,0.081711,0.814376
street_id,-0.004561,1.000000,-0.004698,-0.000971,0.003399,-0.002125,-0.002297,-0.012059,-0.008049,0.013814,...,0.007013,-0.003661,-0.000998,0.021031,0.000250,-0.000540,0.001738,NaN,0.002769,-0.004011
build_tech,-0.002651,-0.004698,1.000000,0.178180,0.318536,0.120148,0.792551,0.015675,0.004575,0.018670,...,0.016051,0.095276,0.039286,0.378693,0.004498,0.003131,0.010180,NaN,0.004591,-0.004197
floor,0.000376,-0.000971,0.178180,1.000000,0.155564,0.004819,0.141292,0.007247,0.002300,-0.005676,...,0.002502,0.072306,0.007603,0.198841,-0.001128,-0.002113,0.002423,NaN,0.005573,0.001513
area,0.001707,0.003399,0.318536,0.155564,1.000000,0.796843,0.251236,0.005507,0.000266,0.162042,...,0.043500,0.024862,0.041993,0.640858,0.008514,0.008232,0.006833,NaN,0.005158,-0.003074
rooms,0.002140,-0.002125,0.120148,0.004819,0.796843,1.000000,0.095090,0.003990,0.002224,0.209774,...,0.011790,-0.006253,0.012622,0.373064,0.007217,0.006632,0.004406,NaN,0.005413,-0.001837
balcon,-0.001096,-0.002297,0.792551,0.141292,0.251236,0.095090,1.000000,0.013207,-0.000883,0.018187,...,0.017068,0.073845,0.028823,0.301219,0.007470,0.007278,0.007956,NaN,0.003056,-0.005020
metro_dist,-0.000467,-0.012059,0.015675,0.007247,0.005507,0.003990,0.013207,1.000000,-0.002081,0.005689,...,0.001574,0.007695,-0.001647,-0.107506,0.000119,-0.000367,-0.006931,NaN,-0.002702,-0.000513
g_lift,0.000224,-0.008049,0.004575,0.002300,0.000266,0.002224,-0.000883,-0.002081,1.000000,-0.001278,...,-0.002770,0.000704,-0.008096,-0.001503,-0.001747,-0.001082,0.000171,NaN,0.000924,0.000910
n_photos,0.003981,0.013814,0.018670,-0.005676,0.162042,0.209774,0.018187,0.005689,-0.001278,1.000000,...,-0.002831,0.002879,0.004468,0.184517,-0.002726,-0.002755,0.000188,NaN,0.003407,0.005308


In [8]:
corr(train)
# buid_tech floor area rooms balcon metro n_photos kw10?

corr id with price: 0.034704301298528153
corr street_id with price: 0.021030980603670008
corr build_tech with price: 0.37869281008167743
corr floor with price: 0.19884113763498718
corr area with price: 0.6408584152059579
corr rooms with price: 0.37306380739546896
corr balcon with price: 0.3012187445540306
corr metro_dist with price: -0.10750592623490601
corr g_lift with price: -0.0015027916872021702
corr n_photos with price: 0.18451739970688474
corr kw1 with price: 0.021692420299724024
corr kw2 with price: -0.054994462147079995
corr kw3 with price: -0.04994026587142809
corr kw4 with price: -0.024334625976328534
corr kw5 with price: -0.015934277717139844
corr kw6 with price: 0.009485512527743916
corr kw7 with price: 0.007960806722624097
corr kw8 with price: 0.002184360884286784
corr kw9 with price: 0.003969804685561741
corr kw10 with price: 0.09984220475916102
corr kw11 with price: 0.038221063012781914
corr kw12 with price: 0.033094563835263754
corr kw13 with price: 0.025286545865287825

In [9]:
show_data(train)

date: ['2011-01-01T00:00:00.000000000' '2011-02-01T00:00:00.000000000'
 '2011-03-01T00:00:00.000000000' '2011-04-01T00:00:00.000000000'
 '2011-05-01T00:00:00.000000000' '2011-06-01T00:00:00.000000000'
 '2011-07-01T00:00:00.000000000' '2011-08-01T00:00:00.000000000'
 '2011-09-01T00:00:00.000000000' '2011-10-01T00:00:00.000000000'
 '2011-11-01T00:00:00.000000000' '2011-12-01T00:00:00.000000000'
 '2012-01-01T00:00:00.000000000' '2012-02-01T00:00:00.000000000'
 '2012-03-01T00:00:00.000000000' '2012-04-01T00:00:00.000000000'
 '2012-05-01T00:00:00.000000000' '2012-06-01T00:00:00.000000000']
build_tech: [ 1.  0. nan  2.]
floor: [ 5  3  1  2  7  6  8 11 10 24  4 12 22  9 16 18 17 25 20 14 21 13 19 23
 15]
area: [ 67  45  61  51  60  34  36  33  54  43  31  58  32  35  50  44  48  59
  52  47 125  88 168  65 167  46  71  63  55  57  90  41  62  38  75  70
  40 183  56 166  39  42  98  74  53  37  29  73 104 142  66  72  95  91
  49  64  87 105  89  30 110  86 163  97  99  69 123  76 112 124 109

In [10]:
show_data(test)

date: ['2012-06-01T00:00:00.000000000' '2012-07-01T00:00:00.000000000'
 '2012-08-01T00:00:00.000000000' '2012-09-01T00:00:00.000000000'
 '2012-10-01T00:00:00.000000000' '2012-11-01T00:00:00.000000000'
 '2012-12-01T00:00:00.000000000' '2013-01-01T00:00:00.000000000'
 '2013-02-01T00:00:00.000000000' '2013-03-01T00:00:00.000000000'
 '2013-04-01T00:00:00.000000000' '2013-05-01T00:00:00.000000000'
 '2013-06-01T00:00:00.000000000' '2013-07-01T00:00:00.000000000'
 '2013-08-01T00:00:00.000000000' '2013-09-01T00:00:00.000000000'
 '2013-10-01T00:00:00.000000000' '2013-11-01T00:00:00.000000000'
 '2013-12-01T00:00:00.000000000']
build_tech: [ 1. nan  0.  2.]
floor: [12  1  3  8  7  4  6  9  2  5 13 22 11 23 20 10 18 14 15 19 25 17 24 21
 16]
area: [ 35  52  41  39  75  54  65 156  57  60  36  51  59  58  31  34  62  33
  44  88  38  61  45  43  50 126  53 110  32  37  46  56  63  49  40  91
  64  48  47  67  92  77 100  99  42  71 101  68  30  73  70  76  89  74
 183  97 167  90  95 154  72 125  8

# 1 МОДЕЛЬ. ПОДГОТОВКА

In [12]:
unnecessary1 = ['id', 'date', 'n_photos', 'day']
y_train1 = train[train.date < '2012-01-01T00:00:00.000000000'][['price']]
y_test1 = train[train.date >= '2012-01-01T00:00:00.000000000'][['price']]
X_train1 = train[train.date < '2012-01-01T00:00:00.000000000']
X_test1 = train[train.date >= '2012-01-01T00:00:00.000000000']
X_train1 = X_train1.drop(unnecessary1, axis=1)
X_train1 = X_train1.drop('price', axis=1)
X_test1 = X_test1.drop(unnecessary1, axis=1)
X_test1 = X_test1.drop('price', axis=1)b
test_1 = test.drop(unnecessary1, axis=1)
print(X_train1.tail())

       street_id  build_tech  floor  area  rooms  balcon  metro_dist  g_lift  \
67004        251         1.0      6    35      1       1        20.0     1.0   
67005         63         1.0      1    60      3       1        10.0     NaN   
67006        262         NaN     12    44      1       2        10.0     0.0   
67007        414         1.0      4    71      2       0        20.0     1.0   
67008         54         0.0      5    45      2       0        20.0     0.0   

       kw1  kw2  ...  kw8  kw9  kw10  kw11  kw12  kw13  quarter  month  week  \
67004    0    1  ...    0    1     0     0     0     0        4     12    48   
67005    0    0  ...    0    0     0     0     0     0        4     12    48   
67006    0    0  ...    0    0     0     0     0     0        4     12    48   
67007    0    0  ...    0    0     0     0     0     0        4     12    48   
67008    0    1  ...    0    0     0     0     0     0        4     12    48   

       dayofweek  
67004          3  


In [13]:
X_train1 = delete_nan(X_train1)
X_test1 = delete_nan(X_test1)
test_1 = delete_nan(test_1)

# 1 МОДЕЛЬ. ПРЕДСКАЗАНИЕ

In [14]:
from sklearn.metrics import mean_squared_error
model1 = xgb.XGBRegressor(random_state=RANDOM_STATE)
model1.fit(X_train1,y_train1)

[23:28:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [105]:
##### НЕ ЗАПУСКАТЬ!!!!!!!!!!!!!
from tqdm import tqdm
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split

best_score = 10000000000000
best_depth = 0.1
best_estimators = 0.1
best_model = None

early_stopping_rounds = 500 # кол-во кругов, после которого без прироста скора заканчивается работа
rounds_without_gain = 0

depth_values = range(3, 21, 1)
estimators_values = range(10, 500, 10)

for depth in tqdm(depth_values):
    for est in tqdm(estimators_values):
        mod = xgb.XGBRegressor(max_depth=depth, n_estimators=est, random_state=RANDOM_STATE)
        score = np.mean(cross_val_score(mod, X_train1, y_train1, cv=5, scoring="neg_mean_squared_error"))
    
        if score < best_score:
            best_score = score
            best_model = mod
            best_depth = depth
            best_estimators = est
            rounds_without_gain = 0
        else:
            rounds_without_gain += 1
        if rounds_without_gain == early_stopping_rounds:
            print("Stopping with {}".format(early_stopping_rounds))
            break
        










  0%|          | 0/18 [00:00<?, ?it/s]







  0%|          | 0/49 [00:00<?, ?it/s]

[19:44:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:44:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:44:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:44:52] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:44:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.










  2%|▏         | 1/49 [00:16<12:56, 16.17s/it]

[19:44:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:45:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:45:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:45:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:45:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.










  4%|▍         | 2/49 [00:32<12:38, 16.14s/it]

...

[23:28:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:28:04] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:28:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:28:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:28:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.











 98%|█████████▊| 48/49 [12:42<00:15, 15.86s/it]

[23:28:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:28:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:28:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:28:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:28:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.











100%|██████████| 49/49 [12:57<00:00, 15.85s/it]






100%|██████████| 18/18 [3:43:50<00:00, 764.11s/it]

Hyperparameters selection finished, best score - -6194237016953.7500 at depth=3, est=10
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=10,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)


In [106]:
print(f"Hyperparameters selection finished, best score - {best_score:0.4f} at depth={best_depth}, est={best_estimators}")
print(best_model)

Hyperparameters selection finished, best score - -6194237016953.7500 at depth=3, est=10
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=10,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)


In [15]:
preds1 = model1.predict(X_test1)
print(preds1)
mse = mean_squared_error(preds1,y_test1)
print(mse)
preds1 = model1.predict(test_1)
print(preds1)

[2900454.  5296868.5 4126632.8 ... 4519929.5 1990061.1 2509321.2]
6651681511180.156
[3853083.8 5734403.5 2797577.5 ... 4315988.5 4496498.  3539686.2]


In [16]:
make_submission(test["id"].values, preds1)

# 2 МОДЕЛЬ. ПОДГОТОВКА

In [17]:
train.columns

Index(['id', 'date', 'street_id', 'build_tech', 'floor', 'area', 'rooms',
       'balcon', 'metro_dist', 'g_lift', 'n_photos', 'kw1', 'kw2', 'kw3',
       'kw4', 'kw5', 'kw6', 'kw7', 'kw8', 'kw9', 'kw10', 'kw11', 'kw12',
       'kw13', 'price', 'quarter', 'month', 'week', 'day', 'dayofweek'],
      dtype='object')

In [18]:
# buid_tech floor area rooms balcon metro n_photos kw10?
unnecessary2 = ['id', 'date', 'street_id', 'g_lift', 'kw1', 'kw2', 'kw3',
       'kw4', 'kw5', 'kw6', 'kw7', 'kw8', 'kw9', 'kw11', 'kw12',
       'kw13', 'quarter', 'day']
y_train2 = train[train.date < '2012-01-01T00:00:00.000000000'][['price']]
y_test2 = train[train.date >= '2012-01-01T00:00:00.000000000'][['price']]
X_train2 = train[train.date < '2012-01-01T00:00:00.000000000']
X_test2 = train[train.date >= '2012-01-01T00:00:00.000000000']
X_train2 = X_train2.drop(unnecessary2, axis=1)
X_train2 = X_train2.drop('price', axis=1)
X_test2 = X_test2.drop(unnecessary2, axis=1)
X_test2 = X_test2.drop('price', axis=1)
test_2 = test.drop(unnecessary2, axis=1)

In [19]:
X_train2 = delete_nan(X_train2)
X_test2 = delete_nan(X_test2)
test_2 = delete_nan(test_2)

In [20]:
####### НЕ ЗАПУСКАТЬ!!!!!!!!!!!!!!!!!!!
%%time
xgbr_params = {'objective':'reg:squarederror',
               'n_jobs':-1, 
               'min_child_weight':1,
               'eta':0.3, 
               'subsample':0.8, 
               'gamma':0.5, 
               'colsample_bytree':0.8, 
               'random_state':RANDOM_STATE
               'learning_rate':0.05}
gs = GridSearchCV(
    xgb.XGBRegressor(xgbr_params, 'objective'='reg:squarederror'),
    param_grid ={
        'max_depth': range(3, 11, 1),
        'n_estimators': range(100, 1201, 100),
    },
    scoring='neg_mean_squared_error', cv=5
)
gs.fit(X_train2, y_train2)



print(gs.best_params_)
print(gs.best_score_)

[22:16:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:16:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:16:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:16:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:16:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:16:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:16:28] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:16:28] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:16:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[22:18:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:18:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:18:21] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:18:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:18:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:18:30] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:18:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:18:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:18:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[22:23:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:23:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:23:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:23:35] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:23:35] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:23:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:23:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:23:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:23:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[22:27:00] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:27:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:27:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:27:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:27:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:27:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:27:35] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:27:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:27:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[22:36:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:37:04] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:37:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:37:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:37:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:37:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:38:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:38:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:38:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[22:43:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:43:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:43:35] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:43:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:43:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:44:04] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:44:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:44:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:44:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[22:59:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:59:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:59:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:00:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:00:35] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:00:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:01:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:01:30] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:01:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[23:09:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:09:28] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:09:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:09:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:10:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:10:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:10:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:10:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:11:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

KeyboardInterrupt: 

# 2 МОДЕЛЬ. ПРЕДСКАЗАНИЕ

In [20]:
model2 = xgb.XGBRegressor(random_state=RANDOM_STATE)
model2.fit(X_train2,y_train2)

[23:28:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [21]:
preds2 = model2.predict(X_test2)
print(preds2)
print(y_test2.head())
print(y_test2.tail())
mse = mean_squared_error(preds2,y_test2)
print(mse)
preds2 = model2.predict(test_2)
print(preds2)

[2538494.2 5681744.5 3867206.2 ... 4282547.  1795350.9 2346254.5]
          price
67009   3527000
67010   5170000
67011   3165000
67012   2504000
67013  11016000
         price
99995  3527000
99996  4166000
99997  4996000
99998  1258000
99999  3507000
7248611627809.838
[5163281.5 5397606.  2223614.5 ... 5294959.  4669958.  3346289.2]


In [22]:
make_submission(test["id"].values, preds2)

# 2 МОДЕЛЬ БЕЗ РАЗДЕЛЕНИЯ ПО ДАТАМ

In [23]:
unnecessary2 = ['id', 'date', 'street_id', 'g_lift', 'kw1', 'kw2', 'kw3',
       'kw4', 'kw5', 'kw6', 'kw7', 'kw8', 'kw9', 'kw11', 'kw12',
       'kw13', 'quarter', 'month', 'week', 'day', 'dayofweek']
y2 = train['price']
X2 = train.drop(unnecessary2, axis=1)
X2 = X2.drop('price', axis=1)
X2 = delete_nan(X2)
test_2 = test.drop(unnecessary2, axis=1)
test_2 = delete_nan(test_2)
print(X2.head())

   build_tech  floor  area  rooms  balcon  metro_dist  n_photos  kw10
0         1.0      5    67      2       1        25.0         1     0
1         0.0      3    45      2       0        20.0         0     0
2         0.0      1    61      3       1        10.0         1     0
3         0.0      2    51      2       1        15.0         2     0
4         1.0      3    60      3       1        25.0         1     0


In [24]:
model2_1 = xgb.XGBRegressor(random_state=RANDOM_STATE)
model2_1.fit(X2,y2)

[23:28:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [25]:
preds2_1 = model2_1.predict(test_2)

print(preds2_1)

[5315570.  5440915.  2318787.2 ... 5254888.  4512631.5 3173236. ]


In [26]:
make_submission(test["id"].values, preds2_1)

# 3 МОДЕЛЬ СО ВСЕМИ КОЛОНКАМИ. ПОДГОТОВКА

In [27]:
# buid_tech floor area rooms balcon metro n_photos kw10?
unnecessary3 = ['id', 'date', 'day']
y_train3 = train[train.date < '2012-01-01T00:00:00.000000000'][['price']]
y_test3 = train[train.date >= '2012-01-01T00:00:00.000000000'][['price']]
X_train3 = train[train.date < '2012-01-01T00:00:00.000000000']
X_test3 = train[train.date >= '2012-01-01T00:00:00.000000000']
X_train3 = X_train3.drop(unnecessary3, axis=1)
X_train3 = X_train3.drop('price', axis=1)
X_test3 = X_test3.drop(unnecessary3, axis=1)
X_test3 = X_test3.drop('price', axis=1)
test_3 = test.drop(unnecessary3, axis=1)
X_train3 = delete_nan(X_train3)
X_test3 = delete_nan(X_test3)
test_3 = delete_nan(test_3)

# 3 МОДЕЛЬ. ПРЕДСКАЗАНИЕ С РАЗНЫМИ ДАТАМИ

In [28]:
model3 = xgb.XGBRegressor(random_state=RANDOM_STATE, learning_rate=0.05)
model3.fit(X_train3, y_train3)

[23:28:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [29]:
preds3 = model3.predict(X_test3)
print(preds3)
print(y_test3.head())
print(y_test3.tail())
mse = mean_squared_error(preds3,y_test3)
print(mse)
preds3 = model3.predict(test_3)
print(preds3)

[2862729.5 5589328.  4154415.5 ... 4421385.5 2043610.4 2624302. ]
          price
67009   3527000
67010   5170000
67011   3165000
67012   2504000
67013  11016000
         price
99995  3527000
99996  4166000
99997  4996000
99998  1258000
99999  3507000
7263081054060.02
[4798943.5 5254278.5 2731475.8 ... 5045226.5 4554094.  3374773.5]


In [30]:
make_submission(test["id"].values, preds3)

# 4 МОДЕЛЬ С НОРМАЛИЗАЦИЕЙ ПРИЗНАКОВ И ВАН-ХОТТТТ

In [31]:
train.head()
show_data(train)

date: ['2011-01-01T00:00:00.000000000' '2011-02-01T00:00:00.000000000'
 '2011-03-01T00:00:00.000000000' '2011-04-01T00:00:00.000000000'
 '2011-05-01T00:00:00.000000000' '2011-06-01T00:00:00.000000000'
 '2011-07-01T00:00:00.000000000' '2011-08-01T00:00:00.000000000'
 '2011-09-01T00:00:00.000000000' '2011-10-01T00:00:00.000000000'
 '2011-11-01T00:00:00.000000000' '2011-12-01T00:00:00.000000000'
 '2012-01-01T00:00:00.000000000' '2012-02-01T00:00:00.000000000'
 '2012-03-01T00:00:00.000000000' '2012-04-01T00:00:00.000000000'
 '2012-05-01T00:00:00.000000000' '2012-06-01T00:00:00.000000000']
build_tech: [ 1.  0. nan  2.]
floor: [ 5  3  1  2  7  6  8 11 10 24  4 12 22  9 16 18 17 25 20 14 21 13 19 23
 15]
area: [ 67  45  61  51  60  34  36  33  54  43  31  58  32  35  50  44  48  59
  52  47 125  88 168  65 167  46  71  63  55  57  90  41  62  38  75  70
  40 183  56 166  39  42  98  74  53  37  29  73 104 142  66  72  95  91
  49  64  87 105  89  30 110  86 163  97  99  69 123  76 112 124 109

In [32]:
unnecessary4 = ['id', 'date', 'day']
y4 = train['price']
X4 = train.drop(unnecessary4, axis=1)
X4 = delete_nan(X4)
test_4 = test.drop(unnecessary4, axis=1)
test_4 = delete_nan(test_4)
print(test_4.head())

   street_id  build_tech  floor  area  rooms  balcon  metro_dist  g_lift  \
0        522         1.0     12    35      1       1        20.0     1.0   
1        577         1.0      1    52      2       1        10.0     1.0   
2        558         0.0      1    41      2       0        15.0     0.0   
3        445         0.0      3    39      2       0        25.0     1.0   
4        365         0.0      8    75      3       0        30.0     1.0   

   n_photos  kw1  ...  kw8  kw9  kw10  kw11  kw12  kw13  quarter  month  week  \
0         6    0  ...    0    0     0     0     0     0        2      6    22   
1         3    0  ...    0    0     0     0     0     0        2      6    22   
2         2    0  ...    0    0     0     0     0     0        2      6    22   
3         0    0  ...    0    0     0     0     0     0        2      6    22   
4         2    0  ...    0    0     0     0     0     0        2      6    22   

   dayofweek  
0          4  
1          4  
2          

In [33]:
categories = ['street_id', 'build_tech', 'floor', 'area', 'rooms',
       'balcon',  'quarter', 'month', 'week', 'dayofweek']
full = pd.concat([X4, test_4], axis=0)
dfs = []
for col in categories:
    df = pd.get_dummies(full[col], prefix=col)
    dfs.append(df)
    
full = pd.concat([full] + dfs, axis=1)
full = full.drop(categories, axis=1)
full.head()

,g_lift,kw1,kw10,kw11,kw12,kw13,kw2,kw3,kw4,kw5,...,week_44,week_48,week_52,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
0,1.0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1,0.0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,1.0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
3,0.0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
4,0.0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [34]:
X4 = full[full.price.notnull()]
test_4 = full[full.price.isnull()]
test_4 = test_4.drop("price", axis=1)
X4 = X4.drop("price", axis=1)
X4.columns
X = X4[X4.columns].values

In [ ]:
model4 = xgb.XGBRegressor(xgbr_params)
score = np.mean(cross_val_score(model4, X, y4, cv=5, scoring="neg_mean_squared_error"))
print(score)

In [35]:
xgbr_params = {'max_depth':3,
               'n_estimators':100,
    'objective':'reg:squarederror',
               'n_jobs':-1, 
               'min_child_weight':1,
               'eta':0.3, 
               'subsample':0.8, 
               'gamma':0.5, 
               'colsample_bytree':0.8, 
               'random_state':RANDOM_STATE,
               'learning_rate':0.05}
model4 = xgb.XGBRegressor(**xgbr_params)
model4.fit(X4, y4)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.3, gamma=0.5,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=-1, nthread=None, objective='reg:squarederror',
             random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.8, verbosity=1)

In [36]:
preds4 = model4.predict(test_4)
print(preds4)

[5071188.  5403640.5 3781876.2 ... 5463691.5 5294247.5 4240210. ]


In [37]:
make_submission(test["id"].values, preds4)

# 5 МОДЕЛЬ. ИНТУИТИВНАЯ

In [8]:
train.head(15)

,id,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,...,kw10,kw11,kw12,kw13,price,quarter,month,week,day,dayofweek
0,0,2011-01-01,511,1.0,5,67,2,1,NaN,1.0,...,0,0,0,0,18473000,1,1,52,1,5
1,1,2011-01-01,299,0.0,3,45,2,0,20.0,0.0,...,0,0,0,0,4157000,1,1,52,1,5
2,2,2011-01-01,217,NaN,1,61,3,1,10.0,1.0,...,0,0,0,0,4530000,1,1,52,1,5
3,3,2011-01-01,212,NaN,2,51,2,1,15.0,0.0,...,0,0,0,0,7981000,1,1,52,1,5
4,4,2011-01-01,160,1.0,3,60,3,1,25.0,0.0,...,0,0,0,0,11888000,1,1,52,1,5
5,5,2011-01-01,438,NaN,2,34,1,0,20.0,0.0,...,0,0,0,0,2357000,1,1,52,1,5
6,6,2011-01-01,404,0.0,3,36,1,0,15.0,NaN,...,0,0,0,0,3841000,1,1,52,1,5
7,7,2011-01-01,410,NaN,7,33,1,0,NaN,0.0,...,0,0,0,0,2736000,1,1,52,1,5
8,8,2011-01-01,196,NaN,6,54,2,1,25.0,NaN,...,0,0,0,0,9047000,1,1,52,1,5
9,9,2011-01-01,331,0.0,3,43,2,0,NaN,1.0,...,0,0,0,0,1577000,1,1,52,1,5


In [9]:
corr(train)

corr id with price: 0.034704301298528153
corr street_id with price: 0.021030980603670008
corr build_tech with price: 0.37869281008167743
corr floor with price: 0.19884113763498718
corr area with price: 0.6408584152059579
corr rooms with price: 0.37306380739546896
corr balcon with price: 0.3012187445540306
corr metro_dist with price: -0.10750592623490601
corr g_lift with price: -0.0015027916872021702
corr n_photos with price: 0.18451739970688474
corr kw1 with price: 0.021692420299724024
corr kw2 with price: -0.054994462147079995
corr kw3 with price: -0.04994026587142809
corr kw4 with price: -0.024334625976328534
corr kw5 with price: -0.015934277717139844
corr kw6 with price: 0.009485512527743916
corr kw7 with price: 0.007960806722624097
corr kw8 with price: 0.002184360884286784
corr kw9 with price: 0.003969804685561741
corr kw10 with price: 0.09984220475916102
corr kw11 with price: 0.038221063012781914
corr kw12 with price: 0.033094563835263754
corr kw13 with price: 0.025286545865287825

In [10]:
train.columns

Index(['id', 'date', 'street_id', 'build_tech', 'floor', 'area', 'rooms',
       'balcon', 'metro_dist', 'g_lift', 'n_photos', 'kw1', 'kw2', 'kw3',
       'kw4', 'kw5', 'kw6', 'kw7', 'kw8', 'kw9', 'kw10', 'kw11', 'kw12',
       'kw13', 'price', 'quarter', 'month', 'week', 'day', 'dayofweek'],
      dtype='object')

In [11]:
unnecessary5 = ['id', 'date', 'street_id', 'floor', 'area', 'rooms',
       'balcon']
y_train5 = train[train.date < '2012-01-01T00:00:00.000000000'][['price']]
y_test5 = train[train.date >= '2012-01-01T00:00:00.000000000'][['price']]
X_train5 = train[train.date < '2012-01-01T00:00:00.000000000']
X_test5 = train[train.date >= '2012-01-01T00:00:00.000000000']
X_train5 = X_train5.drop(unnecessary5, axis=1)
X_train5 = X_train5.drop('price', axis=1)
X_test5 = X_test5.drop(unnecessary5, axis=1)
X_test5 = X_test5.drop('price', axis=1)
test_5 = test.drop(unnecessary5, axis=1)
print(X_train5.tail())

       build_tech  metro_dist  g_lift  n_photos  kw1  kw2  kw3  kw4  kw5  kw6  \
67004         1.0        20.0     1.0         4    0    1    0    0    0    0   
67005         1.0        10.0     NaN         3    0    0    0    0    0    0   
67006         NaN        10.0     0.0         3    0    0    0    0    0    0   
67007         1.0        20.0     1.0         1    0    0    0    0    0    0   
67008         0.0        20.0     0.0         3    0    1    0    0    0    0   

       ...  kw9  kw10  kw11  kw12  kw13  quarter  month  week  day  dayofweek  
67004  ...    1     0     0     0     0        4     12    48    1          3  
67005  ...    0     0     0     0     0        4     12    48    1          3  
67006  ...    0     0     0     0     0        4     12    48    1          3  
67007  ...    0     0     0     0     0        4     12    48    1          3  
67008  ...    0     0     0     0     0        4     12    48    1          3  

[5 rows x 22 columns]


In [12]:
X_train5 = delete_nan(X_train5)
X_test5 = delete_nan(X_test5)
test_5 = delete_nan(test_5)

In [13]:
xgbr_params = {'max_depth':7,
               'n_estimators':300,
    'objective':'reg:squarederror',
               'n_jobs':-1, 
               'min_child_weight':1,
               'eta':0.3, 
               'subsample':0.8, 
               'gamma':0.5, 
               'colsample_bytree':0.8, 
               'random_state':RANDOM_STATE,
               'learning_rate':0.05}
model5 = xgb.XGBRegressor(**xgbr_params)
model5.fit(X_train5, y_train5)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.3, gamma=0.5,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=7, min_child_weight=1, missing=None, n_estimators=300,
             n_jobs=-1, nthread=None, objective='reg:squarederror',
             random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.8, verbosity=1)

In [18]:
preds5 = model5.predict(X_test5)
print(preds5)
print(y_test5.head())
print(y_test5.tail())
mse = mean_squared_error(preds5,y_test5)
print(mse)
preds5 = model5.predict(test_5)
print(preds5)
make_submission(test["id"].values, preds5)

[2981893.8 5316675.  4220504.  ... 4359337.  3262511.  3832277.5]
          price
67009   3527000
67010   5170000
67011   3165000
67012   2504000
67013  11016000
         price
99995  3527000
99996  4166000
99997  4996000
99998  1258000
99999  3507000
12709012687855.707
[7234349.5 6273993.  4164452.8 ... 5441699.5 4734976.5 3193095.5]


In [19]:
y5 = train['price']
X5 = train.drop(unnecessary5, axis=1)
X5 = X5.drop('price', axis=1)
X5 = delete_nan(X5)
test_5 = test.drop(unnecessary5, axis=1)
test_5 = delete_nan(test_5)
print(X5.head())

   build_tech  metro_dist  g_lift  n_photos  kw1  kw2  kw3  kw4  kw5  kw6  \
0         1.0        25.0     1.0         1    0    0    0    0    0    0   
1         0.0        20.0     0.0         0    0    0    0    0    0    0   
2         0.0        10.0     1.0         1    0    1    0    0    0    0   
3         0.0        15.0     0.0         2    0    1    0    0    0    0   
4         1.0        25.0     0.0         1    1    0    0    0    0    0   

   ...  kw9  kw10  kw11  kw12  kw13  quarter  month  week  day  dayofweek  
0  ...    0     0     0     0     0        1      1    52    1          5  
1  ...    0     0     0     0     0        1      1    52    1          5  
2  ...    0     0     0     0     0        1      1    52    1          5  
3  ...    0     0     0     0     0        1      1    52    1          5  
4  ...    0     0     0     0     0        1      1    52    1          5  

[5 rows x 22 columns]


In [20]:
xgbr_params = {'max_depth':5,
               'n_estimators':100,
    'objective':'reg:squarederror',
               'n_jobs':-1, 
               'min_child_weight':1,
               'eta':0.3, 
               'subsample':0.8, 
               'gamma':0.5, 
               'colsample_bytree':0.8, 
               'random_state':RANDOM_STATE,
               'learning_rate':0.05}
model5_1 = xgb.XGBRegressor(**xgbr_params)
model5_1.fit(X5, y5)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.3, gamma=0.5,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=-1, nthread=None, objective='reg:squarederror',
             random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.8, verbosity=1)

In [21]:
preds5_1 = model5_1.predict(test_5)
print(preds5_1)
make_submission(test["id"].values, preds5_1)

[7662182.5 6268467.  4089097.  ... 5530724.  4626954.  3315906.2]


In [22]:
model5_1 = xgb.XGBRegressor(random_state=RANDOM_STATE)
model5_1.fit(X5, y5)

[22:42:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [23]:
preds5_1 = model5_1.predict(test_5)
print(preds5_1)
make_submission(test["id"].values, preds5_1)

[7618127.  6283374.5 4133204.5 ... 5742007.  4678535.5 3267647. ]


In [28]:
####### НЕ ЗАПУСКАТЬ!!!!!!!!!!!!!!!!!!!
#%%time
xgbr_params = {'objective':'reg:squarederror',
               'n_jobs':-1, 
               'min_child_weight':1,
               'eta':0.3, 
               'subsample':0.8, 
               'gamma':0.5, 
               'colsample_bytree':0.8, 
               'random_state':RANDOM_STATE,
               'learning_rate':0.05}
gs = GridSearchCV(
    xgb.XGBRegressor(**xgbr_params),
    param_grid ={
        'max_depth': range(3, 11, 1),
        'n_estimators': range(100, 1201, 100),
    },
    scoring='neg_mean_squared_error', cv=5
)
gs.fit(X5, y5)


KeyboardInterrupt: 

In [ ]:
print(gs.best_params_)
print(gs.best_score_)

# МОДЕЛЬ ПОСЛЕДНЯЯ. ЭКСПЕРИМЕНТАЛЬНАЯ

In [8]:
# ВОТ ЗНАЧИМЫЕ
train.columns

Index(['id', 'date', 'street_id', 'build_tech', 'floor', 'area', 'rooms',
       'balcon', 'metro_dist', 'g_lift', 'n_photos', 'kw1', 'kw2', 'kw3',
       'kw4', 'kw5', 'kw6', 'kw7', 'kw8', 'kw9', 'kw10', 'kw11', 'kw12',
       'kw13', 'price', 'quarter', 'month', 'week', 'day', 'dayofweek',
       'year'],
      dtype='object')

In [22]:
unnecessary = ['id', 'date', 'week', 'dayofweek','day']
y = train['price']
X = train.drop(unnecessary, axis=1)
X = delete_nan(X)
test_ = test.drop(unnecessary, axis=1)
test_ = delete_nan(test_)
print(X.head())

   street_id  build_tech  floor  area  rooms  balcon  metro_dist  g_lift  \
0        511         1.0      5    67      2       1        30.0     1.0   
1        299         0.0      3    45      2       0        20.0     0.0   
2        217         0.0      1    61      3       1        10.0     1.0   
3        212         0.0      2    51      2       1        15.0     0.0   
4        160         1.0      3    60      3       1        25.0     0.0   

   n_photos  kw1  ...  kw8  kw9  kw10  kw11  kw12  kw13     price  quarter  \
0         1    0  ...    0    0     0     0     0     0  18473000        1   
1         0    0  ...    0    0     0     0     0     0   4157000        1   
2         1    0  ...    0    0     0     0     0     0   4530000        1   
3         2    0  ...    0    0     0     0     0     0   7981000        1   
4         1    1  ...    0    0     0     0     0     0  11888000        1   

   month  year  
0      1  2011  
1      1  2011  
2      1  2011  
3     

In [23]:
#########################################################'street_id', 'build_tech', 'area', 'quarter',
categories = [ 'year']
full = pd.concat([X, test_], axis=0)
dfs = []
for col in categories:
    df = pd.get_dummies(full[col], prefix=col)
    dfs.append(df)
    
full = pd.concat([full] + dfs, axis=1)
full = full.drop(categories, axis=1)
full.head()

X = full[full.price.notnull()]
test_ = full[full.price.isnull()]
test_ = test_.drop("price", axis=1)
X = X.drop("price", axis=1)
X.columns

Index(['area', 'balcon', 'build_tech', 'floor', 'g_lift', 'kw1', 'kw10',
       'kw11', 'kw12', 'kw13', 'kw2', 'kw3', 'kw4', 'kw5', 'kw6', 'kw7', 'kw8',
       'kw9', 'metro_dist', 'month', 'n_photos', 'quarter', 'rooms',
       'street_id', 'year_2011', 'year_2012', 'year_2013'],
      dtype='object')

In [24]:
xgbr_params = {'max_depth':7,
               'n_estimators':1700,
    'objective':'reg:squarederror',
               'n_jobs':-1, 
               'min_child_weight':1,
               'eta':0.3, 
               'subsample':0.8, 
               'gamma':0.5, 
               'colsample_bytree':0.8, 
               #'random_state':RANDOM_STATE,
               'learning_rate':0.05}
model = xgb.XGBRegressor(**xgbr_params, random_state=RANDOM_STATE)
model.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.3, gamma=0.5,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=7, min_child_weight=1, missing=None, n_estimators=1700,
             n_jobs=-1, nthread=None, objective='reg:squarederror',
             random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.8, verbosity=1)

In [25]:
# 0.35768
preds = model.predict(test_)
print("preds = {}".format(preds))
make_submission(test["id"].values, preds)

preds = [4591449.  4906803.  3032794.8 ... 3562905.2 3851396.5 2783967.2]


In [ ]:
# 4052046787498.4517 day, date, id
# 4039422218246.933 + all date
# 2366004484122.6597 300 estimators
# 1796689311975.8987 500 estimators
# 1414673169073.7144 700 estimators
# 0944934664148.8589 1200 estimators
# 0800607421265.0493 1500 estimators
# 0740464151103.6611 YEAR
# 0736537194097.9059 QUATER
# !0757249247849.1569! one-hot for year упало, но не сильно
# 1467700354614.0796 one-hot for categories упало
# 0781340545576.4172 one-hot with smallest упало, но не сильно
# 0765044768310.2947 only date упало
# 0548897578875.8711 learning rate 0.1 упало
# 0524592591045.6588 without one-hot упало
# 0747411638871.532 with month
# 0724600356204.6884 one-hot with year + 0 parameters упало
# 0799657768611.9486 one-hot with year + date упало, но не сильно
# 0768989233160.5807 metro 1000000 упало
# 0755011149501.2731 max упало
# 0772856843197.0774 max metro
# 0702991132361.657 1700 estimators
# 0469792449201.0559 7 depth

# МОДЕЛИ НА ПОДСТРАХОВКУ

In [16]:
unnecessary = ['id', 'date', 'week', 'month', 'dayofweek','day']
y = train['price']
X = train.drop(unnecessary, axis=1)
X = delete_nan(X)
test_ = test.drop(unnecessary, axis=1)
test_ = delete_nan(test_)
print(X.head())

categories = [ 'quarter', 'year']
full = pd.concat([X, test_], axis=0)
dfs = []
for col in categories:
    df = pd.get_dummies(full[col], prefix=col)
    dfs.append(df)
    
full = pd.concat([full] + dfs, axis=1)
full = full.drop(categories, axis=1)
full.head()

X = full[full.price.notnull()]
test_ = full[full.price.isnull()]
test_ = test_.drop("price", axis=1)
X = X.drop("price", axis=1)
X.columns

   street_id  build_tech  floor  area  rooms  balcon  metro_dist  g_lift  \
0        511         1.0      5    67      2       1        30.0     1.0   
1        299         0.0      3    45      2       0        20.0     0.0   
2        217         0.0      1    61      3       1        10.0     1.0   
3        212         0.0      2    51      2       1        15.0     0.0   
4        160         1.0      3    60      3       1        25.0     0.0   

   n_photos  kw1  ...  kw7  kw8  kw9  kw10  kw11  kw12  kw13     price  \
0         1    0  ...    0    0    0     0     0     0     0  18473000   
1         0    0  ...    0    0    0     0     0     0     0   4157000   
2         1    0  ...    0    0    0     0     0     0     0   4530000   
3         2    0  ...    0    0    0     0     0     0     0   7981000   
4         1    1  ...    0    0    0     0     0     0     0  11888000   

   quarter  year  
0        1  2011  
1        1  2011  
2        1  2011  
3        1  2011  
4  

Index(['area', 'balcon', 'floor', 'g_lift', 'kw1', 'kw10', 'kw11', 'kw12',
       'kw13', 'kw2', 'kw3', 'kw4', 'kw5', 'kw6', 'kw7', 'kw8', 'kw9',
       'metro_dist', 'n_photos', 'rooms', 'street_id', 'quarter_1',
       'quarter_2', 'quarter_3', 'quarter_4', 'build_tech_0.0',
       'build_tech_1.0', 'build_tech_2.0', 'year_2011', 'year_2012',
       'year_2013'],
      dtype='object')

In [17]:
xgbr_params = {'max_depth':7,
               'n_estimators':1700,
    'objective':'reg:squarederror',
               'n_jobs':-1, 
               'min_child_weight':1,
               'eta':0.3, 
               'subsample':0.8, 
               'gamma':0.5, 
               'colsample_bytree':0.8, 
               #'random_state':RANDOM_STATE,
               'learning_rate':0.05}
model = xgb.XGBRegressor(**xgbr_params, random_state=RANDOM_STATE)
model.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.3, gamma=0.5,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=7, min_child_weight=1, missing=None, n_estimators=1700,
             n_jobs=-1, nthread=None, objective='reg:squarederror',
             random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.8, verbosity=1)

In [18]:
# 0.35722
preds = model.predict(test_)
print(preds)
make_submission(test["id"].values, preds)

[4919244.  4748139.5 3046033.5 ... 3637210.8 4061761.2 2705607.5]


In [19]:
unnecessary = ['id', 'date', 'week', 'month', 'dayofweek','day']
y = train['price']
X = train.drop(unnecessary, axis=1)
X = X.drop('price', axis=1)
X = delete_nan(X)
test_ = test.drop(unnecessary, axis=1)
test_ = delete_nan(test_)
print(X.head())

   street_id  build_tech  floor  area  rooms  balcon  metro_dist  g_lift  \
0        511         1.0      5    67      2       1        30.0     1.0   
1        299         0.0      3    45      2       0        20.0     0.0   
2        217         0.0      1    61      3       1        10.0     1.0   
3        212         0.0      2    51      2       1        15.0     0.0   
4        160         1.0      3    60      3       1        25.0     0.0   

   n_photos  kw1  ...  kw6  kw7  kw8  kw9  kw10  kw11  kw12  kw13  quarter  \
0         1    0  ...    0    0    0    0     0     0     0     0        1   
1         0    0  ...    0    0    0    0     0     0     0     0        1   
2         1    0  ...    0    0    0    0     0     0     0     0        1   
3         2    0  ...    0    0    0    0     0     0     0     0        1   
4         1    1  ...    0    0    0    0     0     0     0     0        1   

   year  
0  2011  
1  2011  
2  2011  
3  2011  
4  2011  

[5 rows x 24 

In [20]:
xgbr_params = {'max_depth':7,
               'n_estimators':1700,
    'objective':'reg:squarederror',
               'n_jobs':-1, 
               'min_child_weight':1,
               'eta':0.3, 
               'subsample':0.8, 
               'gamma':0.5, 
               'colsample_bytree':0.8, 
               #'random_state':RANDOM_STATE,
               'learning_rate':0.05}
model = xgb.XGBRegressor(**xgbr_params, random_state=RANDOM_STATE)
model.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.3, gamma=0.5,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=7, min_child_weight=1, missing=None, n_estimators=1700,
             n_jobs=-1, nthread=None, objective='reg:squarederror',
             random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.8, verbosity=1)

In [21]:
# 0.35597
preds = model.predict(test_)
print(preds)
make_submission(test["id"].values, preds)

[5032441.  4866192.5 2965810.2 ... 3702094.  4080367.2 2952042. ]
